In [0]:
!pip install wget

  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9672 sha256=ad852c2e07530f90e0b8a20c636b0116824536b2913ec993175c5ed0b321d88e
  Stored in directory: /root/.cache/pip/wheels/04/5f/3e/46cc37c5d698415694d83f607f833f83f0149e49b3af9d0f38
Successfully built wget
You should consider upgrading via the '/local_disk0/.ephemeral_nfs/envs/pythonEnv-96cf32c7-05aa-4ca3-9819-08a88d13e9a5/bin/python -m pip install --upgrade pip' command.


In [0]:
%sh
wget https://datarepo.eng.ucsd.edu/mcauley_group/data/amazon_v2/metaFiles2/meta_Books.json.gz

--2024-01-28 02:34:36--  https://datarepo.eng.ucsd.edu/mcauley_group/data/amazon_v2/metaFiles2/meta_Books.json.gz
Resolving datarepo.eng.ucsd.edu (datarepo.eng.ucsd.edu)... 132.239.8.30
Connecting to datarepo.eng.ucsd.edu (datarepo.eng.ucsd.edu)|132.239.8.30|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1219104464 (1.1G) [application/x-gzip]
Saving to: ‘meta_Books.json.gz’

     0K .......... .......... .......... .......... ..........  0%  885K 22m26s
    50K .......... .......... .......... .......... ..........  0% 1.65M 17m5s
   100K .......... .......... .......... .......... ..........  0% 1.68M 15m15s
   150K .......... .......... .......... .......... ..........  0% 1.72M 14m15s
   200K .......... .......... .......... .......... ..........  0% 1.76M 13m36s
   250K .......... .......... .......... .......... ..........  0% 6.25M 11m51s
   300K .......... .......... .......... .......... ..........  0% 2.28M 11m22s
   350K .......... .......... ......

In [0]:
%sh
ls

azure
conf
derby.log
eventlogs
ganglia
hadoop_accessed_config.lst
logs
meta_Books.json.gz
metastore_db
preload_class.lst


In [0]:
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [0]:
spark = SparkSession.builder.appName("Books").getOrCreate()
json_file_path = "file:/databricks/driver/meta_Books.json.gz"
chunk_size = 40000
df1 = spark.read.option("maxFilesPerTrigger", chunk_size).json(json_file_path)

df1.show()

+--------------------+--------------------+----------+--------------------+--------------------+----+--------------------+-------+-------+---+--------+---------------+--------+-------+--------------------+------------+-----+-----+--------------------+
|            also_buy|           also_view|      asin|               brand|            category|date|         description|details|feature|fit|imageURL|imageURLHighRes|main_cat|  price|                rank|similar_item|tech1|tech2|               title|
+--------------------+--------------------+----------+--------------------+--------------------+----+--------------------+-------+-------+---+--------+---------------+--------+-------+--------------------+------------+-----+-----+--------------------+
|[0669009075, B000...|[0019777701, B000...|0000092878|        Keith Graham|                  []|    |[It is a biology ...|   null|     []|   |      []|             []|   Books| $39.94|1,349,781 in Books (|            |     |     |Biology Gods L

In [0]:
df = spark.read.json(json_file_path)
display(df.show())

+--------------------+--------------------+----------+--------------------+--------------------+----+--------------------+-------+-------+---+--------+---------------+--------+-------+--------------------+------------+-----+-----+--------------------+
|            also_buy|           also_view|      asin|               brand|            category|date|         description|details|feature|fit|imageURL|imageURLHighRes|main_cat|  price|                rank|similar_item|tech1|tech2|               title|
+--------------------+--------------------+----------+--------------------+--------------------+----+--------------------+-------+-------+---+--------+---------------+--------+-------+--------------------+------------+-----+-----+--------------------+
|[0669009075, B000...|[0019777701, B000...|0000092878|        Keith Graham|                  []|    |[It is a biology ...|   null|     []|   |      []|             []|   Books| $39.94|1,349,781 in Books (|            |     |     |Biology Gods L

In [0]:
df.toPandas()

In [0]:
uniqueasin = df1.select("asin").distinct()
asins = uniqueasin.collect()

In [0]:
asins

Out[36]: [Row(asin='0001714422'),
 Row(asin='0001842439'),
 Row(asin='0002113643'),
 Row(asin='0002151731'),
 Row(asin='0002168383'),
 Row(asin='0002216973'),
 Row(asin='0002217236'),
 Row(asin='0002234947'),
 Row(asin='0002230534'),
 Row(asin='0002261367'),
 Row(asin='000255111X'),
 Row(asin='0004708237'),
 Row(asin='0004708075'),
 Row(asin='0004702751'),
 Row(asin='000611640X'),
 Row(asin='0006162754'),
 Row(asin='0006476155'),
 Row(asin='0006479529'),
 Row(asin='0006541151'),
 Row(asin='0006544150'),
 Row(asin='0006550479'),
 Row(asin='0006726372'),
 Row(asin='0006925227'),
 Row(asin='0006943543'),
 Row(asin='0007102712'),
 Row(asin='000711981X'),
 Row(asin='0007166710'),
 Row(asin='0007163932'),
 Row(asin='000717201X'),
 Row(asin='0007178484'),
 Row(asin='000718025X'),
 Row(asin='0007219040'),
 Row(asin='0007236093'),
 Row(asin='0007243626'),
 Row(asin='0007254970'),
 Row(asin='0007255349'),
 Row(asin='000727033X'),
 Row(asin='000727596X'),
 Row(asin='0007282567'),
 Row(asin='00073

In [0]:
cols = ['category', 'title', 'brand', 'price', 'asin', 'imageURL', 'imageURLHighRes']
filter_df = df1.select(cols)
display(filter_df.show())

+--------------------+--------------------+--------------------+-------+----------+--------+---------------+
|            category|               title|               brand|  price|      asin|imageURL|imageURLHighRes|
+--------------------+--------------------+--------------------+-------+----------+--------+---------------+
|                  []|Biology Gods Livi...|        Keith Graham| $39.94|0000092878|      []|             []|
|[Books, New, Used...|Mksap 16 Audio Co...|                 Acp|       |000047715X|      []|             []|
|[Books, Arts & Ph...|Flex! Discography...|    Burkhard Jarisch|$199.99|0000004545|      []|             []|
|[Books, Arts & Ph...|Heavenly Highway ...|       Stamps/Baxter|       |0000013765|      []|             []|
|                  []|Georgina Goodman ...|                    |$164.10|0000000116|      []|             []|
|[Books, New, Used...|Principles of Ana...|American Pain Soc...|       |0000555010|      []|             []|
|[Books, Medical B.

In [0]:
filter_df = filter_df.withColumn('imageURL', col('imageURL').cast("string"))
filter_df_with_image_not_empty = filter_df.where("imageURL != '[]'")
filter_df_with_image_not_empty.show()

+--------------------+--------------------+--------------------+-------+----------+--------------------+--------------------+
|            category|               title|               brand|  price|      asin|            imageURL|     imageURLHighRes|
+--------------------+--------------------+--------------------+-------+----------+--------------------+--------------------+
|[Books, Mystery, ...|  The Vulgar Boatman|       Fontana Press|       |0006177611|[https://images-n...|[https://images-n...|
|[Books, Romance, ...|Midnight Is A Lon...|       HarperCollins|       |0007645783|[https://images-n...|[https://images-n...|
|[Books, Science &...|Connecting Math C...|McGraw-Hill Educa...| $99.00|0026846713|[https://images-n...|[https://images-n...|
|[Books, Travel, U...|Following Atticus...|Partners/West Boo...| $12.49|0061997102|[https://images-n...|[https://images-n...|
|[Books, Reference...|Tasks and Communi...|McGraw-Hill Human...|       |0072310545|[https://images-n...|[https://image

In [0]:
filter_df_with_image_not_empty = filter_df_with_image_not_empty.withColumn("price", col("price").substr(2, 100).cast('float'))
filter_df_with_image_not_empty.show()

+--------------------+--------------------+--------------------+-----+----------+--------------------+--------------------+
|            category|               title|               brand|price|      asin|            imageURL|     imageURLHighRes|
+--------------------+--------------------+--------------------+-----+----------+--------------------+--------------------+
|[Books, Mystery, ...|  The Vulgar Boatman|       Fontana Press| null|0006177611|[https://images-n...|[https://images-n...|
|[Books, Romance, ...|Midnight Is A Lon...|       HarperCollins| null|0007645783|[https://images-n...|[https://images-n...|
|[Books, Science &...|Connecting Math C...|McGraw-Hill Educa...|  9.0|0026846713|[https://images-n...|[https://images-n...|
|[Books, Travel, U...|Following Atticus...|Partners/West Boo...| 2.49|0061997102|[https://images-n...|[https://images-n...|
|[Books, Reference...|Tasks and Communi...|McGraw-Hill Human...| null|0072310545|[https://images-n...|[https://images-n...|
|[Books,

In [0]:
filter_df_with_image_not_empty = filter_df_with_image_not_empty.dropna(subset=['price'])
filter_df_with_image_not_empty.show()

+--------------------+--------------------+--------------------+-----+----------+--------------------+--------------------+
|            category|               title|               brand|price|      asin|            imageURL|     imageURLHighRes|
+--------------------+--------------------+--------------------+-----+----------+--------------------+--------------------+
|[Books, Science &...|Connecting Math C...|McGraw-Hill Educa...|  9.0|0026846713|[https://images-n...|[https://images-n...|
|[Books, Travel, U...|Following Atticus...|Partners/West Boo...| 2.49|0061997102|[https://images-n...|[https://images-n...|
|[Books, Education...|Educational Psych...|             Pearson|72.34|0132782731|[https://images-n...|[https://images-n...|
|[Books, Education...|MyEducationLab St...|        eSciencelabs| 0.84|0135140870|[https://images-n...|[https://images-n...|
|[Books, Arts &amp...|Advance Music A C...|       Advance Music| 2.95|0206309341|[https://images-n...|[https://images-n...|
|[Books,

In [0]:
filter_df_with_image_not_empty.createOrReplaceTempView("FilterDF")

In [0]:
%sql
select * from FilterDF

category,title,brand,price,asin,imageURL,imageURLHighRes
"List(Books, Science & Math, Mathematics)","Connecting Math Concepts, Answer Key, Level C",McGraw-Hill Education,9.0,0026846713,[https://images-na.ssl-images-amazon.com/images/I/41A6Aer8ZyL._SS40_.jpg],List(https://images-na.ssl-images-amazon.com/images/I/41A6Aer8ZyL.jpg)
"List(Books, Travel, United States, Northeast, New England)","Following Atticus - Forty-Eight Peaks, One Little Dog and an Extraordinary Friendship",Partners/West Book Distribution,2.49,0061997102,"[https://images-na.ssl-images-amazon.com/images/I/51IMTfG9hPL._SS40_.jpg, https://images-na.ssl-images-amazon.com/images/I/51ONgTLHM2L._SS40_.jpg, https://images-na.ssl-images-amazon.com/images/I/31785bzFasL._SS40_.jpg]","List(https://images-na.ssl-images-amazon.com/images/I/51IMTfG9hPL.jpg, https://images-na.ssl-images-amazon.com/images/I/51ONgTLHM2L.jpg, https://images-na.ssl-images-amazon.com/images/I/31785bzFasL.jpg)"
"List(Books, Education & Teaching, Schools & Teaching, Education Theory, Educational Psychology)",Educational Psychology Windows on Classrooms (Instructor's Review Copy),Pearson,72.34,0132782731,[https://images-na.ssl-images-amazon.com/images/I/51LeNNvIf1L._SS40_.jpg],List(https://images-na.ssl-images-amazon.com/images/I/51LeNNvIf1L.jpg)
"List(Books, Education & Teaching, Schools & Teaching, Certification & Development)",MyEducationLab Student Access Code Card,eSciencelabs,0.84,0135140870,"[https://images-na.ssl-images-amazon.com/images/I/41XQcSz4FeL._SX38_SY50_CR,0,0,38,50_.jpg, https://images-na.ssl-images-amazon.com/images/I/414In%2B-kuOL._SX38_SY50_CR,0,0,38,50_.jpg, https://images-na.ssl-images-amazon.com/images/I/31qdCF9fN7L._SX38_SY50_CR,0,0,38,50_.jpg]","List(https://images-na.ssl-images-amazon.com/images/I/41XQcSz4FeL.jpg, https://images-na.ssl-images-amazon.com/images/I/414In%2B-kuOL.jpg, https://images-na.ssl-images-amazon.com/images/I/31qdCF9fN7L.jpg)"
"List(Books, Arts & Photography, Music, Songbooks, Piano)",Advance Music A Creative Approach to Jazz Piano Harmony,Advance Music,2.95,0206309341,"[https://images-na.ssl-images-amazon.com/images/I/41PH2ngtwuL._SS40_.jpg, https://images-na.ssl-images-amazon.com/images/I/41p%2BL-02qzL._SS40_.jpg]","List(https://images-na.ssl-images-amazon.com/images/I/41PH2ngtwuL.jpg, https://images-na.ssl-images-amazon.com/images/I/41p%2BL-02qzL.jpg)"
"List(Books, Reference, Writing, Research & Publishing Guides, Writing, Travel)",Search for Shambhala,Jonathan Cape,0.03,0224036505,[https://images-na.ssl-images-amazon.com/images/I/51y7ML8ZyCL._SS40_.jpg],List(https://images-na.ssl-images-amazon.com/images/I/51y7ML8ZyCL.jpg)
"List(Books, Computers & Technology, Computer Science, Systems Analysis & Design)",Electronic Circuit Analysis and Design (Mcgraw-Hill Series in Electrical and Computer Engineering),Richard d Irwin,7.0,0256261156,[https://images-na.ssl-images-amazon.com/images/I/518BH2T36YL._SS40_.jpg],List(https://images-na.ssl-images-amazon.com/images/I/518BH2T36YL.jpg)
"List(Books, Romance, Contemporary)",Bride at Whangatapu (Best seller romance),Mills & Boon,3.58,0263746143,[https://images-na.ssl-images-amazon.com/images/I/21nPr23KCOL._SS40_.jpg],List(https://images-na.ssl-images-amazon.com/images/I/21nPr23KCOL.jpg)
"List(Books, Literature & Fiction, Genre Fiction, Historical)",My Lord De Burgh (Historical Romance),Harlequin Mills & Boon,24.61,0263831183,[https://images-na.ssl-images-amazon.com/images/I/518Q59PYCJL._SS40_.jpg],List(https://images-na.ssl-images-amazon.com/images/I/518Q59PYCJL.jpg)
"List(Books, Romance, Contemporary)",A Soldier's Secret (Mills & Boon Special Edition),"Harlequin, Mills & Boon",5.78,0263870391,[https://images-na.ssl-images-amazon.com/images/I/513o1zRpkaL._SS40_.jpg],List(https://images-na.ssl-images-amazon.com/images/I/513o1zRpkaL.jpg)


In [0]:
filter_df_with_image_not_empty1 = filter_df_with_image_not_empty.withColumn('main_category', split(col('category').cast('string'), ",")[0]).withColumn('sub_category', split(col('category').cast('string'), ",")[1])
filter_df_with_image_not_empty1.show()

+--------------------+--------------------+--------------------+-----+----------+--------------------+--------------------+-------------+--------------------+
|            category|               title|               brand|price|      asin|            imageURL|     imageURLHighRes|main_category|        sub_category|
+--------------------+--------------------+--------------------+-----+----------+--------------------+--------------------+-------------+--------------------+
|[Books, Science &...|Connecting Math C...|McGraw-Hill Educa...|  9.0|0026846713|[https://images-n...|[https://images-n...|       [Books|  Science &amp; Math|
|[Books, Travel, U...|Following Atticus...|Partners/West Boo...| 2.49|0061997102|[https://images-n...|[https://images-n...|       [Books|              Travel|
|[Books, Education...|Educational Psych...|             Pearson|72.34|0132782731|[https://images-n...|[https://images-n...|       [Books| Education &amp; ...|
|[Books, Education...|MyEducationLab St...|   

In [0]:
filter_df_with_image_not_empty1 = filter_df_with_image_not_empty1.where("sub_category like '%Computer%'")
filter_df_with_image_not_empty1.show()

+--------------------+--------------------+-------------------+-----+----------+--------------------+--------------------+-------------+--------------------+
|            category|               title|              brand|price|      asin|            imageURL|     imageURLHighRes|main_category|        sub_category|
+--------------------+--------------------+-------------------+-----+----------+--------------------+--------------------+-------------+--------------------+
|[Books, Computers...|Electronic Circui...|    Richard d Irwin|  7.0|0256261156|[https://images-n...|[https://images-n...|       [Books| Computers &amp; ...|
|[Books, Computers...|Munchkin 3 - Cler...|Steve Jackson Games| 0.99|1556347138|[https://images-n...|[https://images-n...|       [Books| Computers &amp; ...|
|[Books, Computers...|SAS SQL 1: Essent...|      SAS Institute|25.82|1607642425|[https://images-n...|[https://images-n...|       [Books| Computers &amp; ...|
|[Books, Computers...|Moleskine 2012 12...|         

In [0]:
@pandas_udf(DoubleType())
def split_columns(category: pd.Series) -> pd.Series:
    iterations = len(category)
    return pd.Series([category.mean()]*iterations)

filter_df_with_image_not_empty1 = filter_df_with_image_not_empty1.withColumn('meanPrice', split_columns(col('price')))
filter_df_with_image_not_empty1.show()

+--------------------+--------------------+-------------------+-----+----------+--------------------+--------------------+-------------+--------------------+-----------------+
|            category|               title|              brand|price|      asin|            imageURL|     imageURLHighRes|main_category|        sub_category|        meanPrice|
+--------------------+--------------------+-------------------+-----+----------+--------------------+--------------------+-------------+--------------------+-----------------+
|[Books, Computers...|Electronic Circui...|    Richard d Irwin|  7.0|0256261156|[https://images-n...|[https://images-n...|       [Books| Computers &amp; ...|8.741999626159668|
|[Books, Computers...|Munchkin 3 - Cler...|Steve Jackson Games| 0.99|1556347138|[https://images-n...|[https://images-n...|       [Books| Computers &amp; ...|8.741999626159668|
|[Books, Computers...|SAS SQL 1: Essent...|      SAS Institute|25.82|1607642425|[https://images-n...|[https://images-n..